# CASE Statement

**Applies to:** Databricks Runtime 16.3+

- **Simple:** `CASE expr WHEN opt THEN stmt ... [ELSE stmt] END CASE` – first match wins.
- **Searched:** `CASE WHEN cond THEN stmt ... [ELSE stmt] END CASE` – first condition true wins.

Use only inside a compound statement. For expressions that return a value, use the CASE *expression* (in SELECT, etc.).

In [0]:
USE CATALOG healthcare_dev;
USE SCHEMA gold;

## Simple CASE: branch on claim type (using variable from query)

In [0]:
BEGIN
  DECLARE typ STRING DEFAULT 'Outpatient';
  IF (SELECT COUNT(*) FROM fact_cms_inpatient_claims) > 0 THEN SET typ = 'Inpatient'; END IF;
  CASE typ
    WHEN 'Inpatient' THEN SELECT 'Inpatient claims' AS description;
    WHEN 'Outpatient' THEN SELECT 'Outpatient claims' AS description;
    ELSE SELECT 'Other claim type' AS description;
  END CASE;
END

## Searched CASE: branch on total claim amount tier

In [0]:
BEGIN
  DECLARE total DECIMAL(18,2) DEFAULT (SELECT COALESCE(SUM(clm_pmt_amt), 0) FROM fact_cms_outpatient_claims) + (SELECT COALESCE(SUM(clm_pmt_amt), 0) FROM fact_cms_inpatient_claims);
  CASE
    WHEN total < 1000 THEN SELECT 'Low' AS tier, total;
    WHEN total < 100000 THEN SELECT 'Medium' AS tier, total;
    WHEN total < 1000000 THEN SELECT 'High' AS tier, total;
    ELSE SELECT 'Very high' AS tier, total;
  END CASE;
END